In [ ]:
!pip install langchain
!pip install langchain-community
!pip install llama-parse
!pip install chromadb
!pip install python-dotenv
!pip install langchain-groq
!pip install chainlit
!pip install fastembed
!pip install unstructured[md]
!pip install markdownify
!pip install langchain-openai
!pip install bs4
!pip install --user -U nltk
!pip install pi_heif
!pip install unstructured unstructured-inference langchain openai chromadb
! pip install langchain unstructured[all-docs] pydantic lxml langchainhub fastapi kaleido uvicorn
!pip install --upgrade lxml pydantic unstructured
!sudo apt-get install poppler-utils tesseract-ocr
!pip install langchain openai weaviate-client ragas
!pip install openai
!pip install chromadb tiktoken
!pip install tiktoken
!pip install datasets

import nltk
nltk.download('all')

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 28.0 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 10.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain lxml
#!pip install "httpx==0.27.2"
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet
import os
os.kill(os.getpid(), 9)

In [ ]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

from llama_parse import LlamaParse
from langchain_community.document_loaders import BSHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
#
from groq import Groq
from langchain_groq import ChatGroq
import joblib
import os
import nest_asyncio  # noqa: E402
nest_asyncio.apply()
from urllib.parse import parse_qsl, urljoin, urlparse, urlunparse
import os
import argparse
import requests
import bs4
#from markdownify import MarkdownConverter
import os
import glob
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
import shutil


In [ ]:
from google.colab import userdata
import os
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-p5I6SOUjzj6H0rT31f1c1PTZ2Xm436P73qc3nSK8CAhZW0vR"
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# all imports
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Speak Now...")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  return b #byte stream
# Save the recorded audio to a .wav file
file_name = "/content/output.wav"

# Get the byte stream
audio_data = record(sec=3)

# Save the byte stream to a .wav file
with open(file_name, "wb") as f:
    f.write(audio_data)

print(f"Audio saved as {file_name}")

In [ ]:
from openai import OpenAI
client = OpenAI()

audio_file= open("/content/output.wav", "rb")
transcription = client.audio.transcriptions.create(
model="whisper-1",
file=audio_file
)
print(transcription.text)

In [ ]:
csv_file_paths = ['medquad.csv']

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders.csv_loader import UnstructuredCSVLoader
import shutil
from langchain_community.document_loaders.csv_loader import CSVLoader

def create_vector_database(file_paths, persist_directory="chroma_db"):

    # Check if the persist directory exists, and clear it if it does
    if os.path.exists(persist_directory):
        shutil.rmtree(persist_directory)  # This clears the previous collection

    all_documents = []  # To store parsed documents

    for file_path in file_paths:
        # Load the markdown document
        loader = UnstructuredCSVLoader(file_path)
        documents = loader.load()
        all_documents.extend(documents)  # Add the loaded documents to the list

    # Split loaded documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=5)
    docs = text_splitter.split_documents(all_documents)

    print(f"Length of documents loaded: {len(all_documents)}")
    print(f"Total number of document chunks generated: {len(docs)}")

    # Initialize Embeddings
    #embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", truncate_dim=384)
    #embed_model = SentenceTransformer('all-mpnet-base-v2')
    embed_model = OpenAIEmbeddings()
    # Create and persist a Chroma vector database from the , truncate_dim=384chunked documents
    vs = Chroma.from_documents(
        documents=docs,
        embedding=embed_model,
        persist_directory=persist_directory,
        collection_name="rag"
    )

    print('Vector DB created successfully!')
    return vs, embed_model


In [ ]:
vs, embed_model= create_vector_database(csv_file_paths)

In [ ]:

chat_model = ChatGroq(temperature=0,
                      model_name="llama3-70b-8192",
                      api_key="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",)



In [ ]:

vectorstore = Chroma(embedding_function=embed_model,
                      persist_directory="chroma_db",
                      collection_name="rag")
 #
retriever=vectorstore.as_retriever(search_kwargs={'k': 5})

In [ ]:

custom_prompt_template = """
Use **only** the following pieces of information to answer the user's question.
If the answer is not available in the provided context, state clearly that you don't know the answer.
Avoid using general knowledge or information that is not present in the provided documents.

Context: {context}
Question: {question}

Only return the helpful answer that exists within the context below and nothing else.
Helpful answer:
"""


In [ ]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval in the healthcare chatbot.
    """
    custom_prompt_template = """
    You are a helpful and knowledgeable AI healthcare assistant specializing in medical conditions.
    Use the provided context to answer the user's question accurately.
    Do not fabricate information. If the answer is not in the context, respond with "I'm sorry, I don't have enough information to answer that."

    Context:
    {context}

    Question:
    {question}

    Ensure your response is clear, concise, and helpful.
    Answer:
    """
    prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context', 'question'])
    return prompt

# Usage
prompt = set_custom_prompt()
prompt


In [ ]:

qa = RetrievalQA.from_chain_type(llm=chat_model,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs={"prompt": prompt})


In [ ]:

def query_vector_database(query):
    # Use the RetrievalQA instance to get the response
    response = qa.invoke(query)

    # Extract the helpful answer from the response
    answer = response['result']

    # Print the response and the source documents
    print(f"Query: {query}")
    print("Answer:")
    print(answer)

    print("\nRetrieved Chunks:")
    for idx, doc in enumerate(response['source_documents']):
        print(f"Chunk {idx + 1}:\n{doc.page_content}\n")

    return response


In [ ]:
query_vector_database("What is heart disease?")

In [ ]:
def get_answer(query):
    response = qa.invoke(query)
    answer = response['result']
    return answer

In [ ]:
!pip install gradio